In [ ]:
pip install boto3

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
import pytz
import warnings
import os
import time
import boto3
warnings.filterwarnings("ignore")
import plotly.graph_objects as go
access_key = '#AWS Access Key#'
secret_key = '#AWS Secret Key# '
region_id = '#AWS RegionID#'
bucket_name = '#AWS Bucketname#'
file_name = f'{file_in_the_bucket}.csv'
ist = pytz.timezone('Asia/Kolkata')
symbol_list_nifty={'HDFCBANK.NS':25.27,'ICICIBANK.NS':24.95,'SBIN.NS':12.04,'AXISBANK.NS':10.09,
                   'KOTAKBANK.NS':9.19,'BANKBARODA.NS':3.15,'PNB.NS':2.31,'INDUSINDBK.NS':6.21,'AUBANK.NS':1.75
                   ,'FEDERALBNK.NS':2.14,'BANDHANBNK.NS':0.94,'IDFCFIRSTB.NS':1.97}
def calculate_percentage(row):
  global symbol_list_nifty
  stock = row['Stock']
  if stock in symbol_list_nifty:
    return symbol_list_nifty[stock]
  else:
    return None
sum_of_values = sum(symbol_list_nifty.values())
api_df = pd.DataFrame(columns=['Datetime', 'Close','Stock'])
for symbol in symbol_list_nifty.keys():
  end_date = datetime.now() + timedelta(days=1)
  end_date_str = end_date.strftime('%Y-%m-%d')
  start_date = end_date - timedelta(days=1)
  start_date_str = start_date.strftime('%Y-%m-%d')
  data = yf.download(symbol, interval='5m', start=start_date_str, end=end_date_str, progress=False)
  data = data[data['Volume'] != 0]
  data.index = data.index.tz_localize(None)
  data.reset_index(inplace=True)
  data=data[['Datetime','Close']]
  data['Datetime'] += pd.Timedelta(minutes=30)
  data['Stock']=symbol
  api_df = pd.concat([api_df, data], ignore_index=True, axis=0)
s3 = boto3.client('s3', aws_access_key_id=access_key,
                  aws_secret_access_key=secret_key,
                  region_name=region_id)
s3.head_object(Bucket=bucket_name, Key=file_name)
obj = s3.get_object(Bucket=bucket_name, Key=file_name)
df = pd.read_csv(obj['Body'])
df = df.reset_index(drop=True)
df=df.sort_values(by='Datetime')
df['Datetime'] = df['Datetime'].astype(str)
df=df[df['Stock']!='RELIANCE.NS']
api_df['Datetime'] = api_df['Datetime'].astype(str)
final_df = pd.merge(df, api_df, on=['Datetime','Stock'], how='left')
final_df['Percentage'] = final_df.apply(calculate_percentage, axis=1)
final_df['Percentage']=final_df['Percentage']/sum_of_values
final_df['Val']=(final_df['Prediction']/final_df['Close']-1)*final_df['Percentage']
nifty_df = final_df.groupby('Datetime').agg({'Val': 'sum'}).reset_index()
symbol='^NSEBANK'
raw_nifty = yf.download(symbol, interval='5m', start=start_date_str, end=end_date_str, progress=False)
raw_nifty.index = raw_nifty.index.tz_localize(None)
raw_nifty.reset_index(inplace=True)
raw_nifty=raw_nifty[['Datetime','Close']]
raw_nifty['Datetime'] += pd.Timedelta(minutes=30)
raw_nifty['Datetime'] = raw_nifty['Datetime'].astype(str)
nifty_df=pd.merge(nifty_df, raw_nifty, on=['Datetime'], how='left')
nifty_df['Val']=nifty_df['Val']+1
nifty_df['Final']=nifty_df['Val']*nifty_df['Close']
nifty_df=nifty_df[['Datetime','Final']]
nifty_df = nifty_df.rename(columns={'Final': 'Close'})
nifty_df['Flag']='FUTURE'
raw_nifty['Flag']='PAST'
nifty_df['Datetime'] = pd.to_datetime(nifty_df['Datetime'], errors='coerce')
raw_nifty['Datetime'] = pd.to_datetime(raw_nifty['Datetime'])
raw_nifty['Datetime'] -= pd.Timedelta(minutes=30)
final=pd.concat([raw_nifty, nifty_df])
current_date = datetime.now().date()#- timedelta(days=1)
final_current_date = final[final['Datetime'].dt.date == current_date]
final_current_date['Time'] = final_current_date['Datetime'].dt.time
final_current_date.drop(columns=['Datetime'], inplace=True)
fig = go.Figure()
past_data = final_current_date[final_current_date['Flag'] == 'PAST']
fig.add_trace(go.Scatter(x=past_data['Time'], y=past_data['Close'], mode='lines', name='PAST'))
future_data = final_current_date[final_current_date['Flag'] == 'FUTURE']
fig.add_trace(go.Scatter(x=future_data['Time'], y=future_data['Close'], mode='lines', name='FUTURE'))
fig.update_layout(title='Line Chart for PAST and FUTURE Trend', xaxis_title='Time', yaxis_title='Close Price')
fig.update_layout(
    title='Line Chart for PAST and FUTURE Trend',
    xaxis_title='Time',
    yaxis_title='Close Price',
    yaxis=dict(
        tickformat='.2f'
    )
)
fig.show()


In [ ]:
s3.head_object(Bucket=bucket_name, Key=file_name)
obj = s3.get_object(Bucket=bucket_name, Key=file_name)
df = pd.read_csv(obj['Body'])
df = df.reset_index(drop=True)
df=df.sort_values(by='Datetime')
df['Datetime'] = df['Datetime'].astype(str)
df=df[df['Stock']=='RELIANCE.NS']
df = df.rename(columns={'Prediction': 'Close'})
end_date = datetime.now() + timedelta(days=1)
end_date_str = end_date.strftime('%Y-%m-%d')
start_date = end_date - timedelta(days=1)
start_date_str = start_date.strftime('%Y-%m-%d')
data = yf.download('RELIANCE.NS', interval='5m', start=start_date_str, end=end_date_str, progress=False)
data = data[data['Volume'] != 0]
data.index = data.index.tz_localize(None)
data.reset_index(inplace=True)
data=data[['Datetime','Close']]
df=df[['Datetime','Close']]
data['Flag']='PAST'
df['Flag']='FUTURE'
data['Datetime'] = pd.to_datetime(data['Datetime'])
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
final=pd.concat([data, df])
current_date = datetime.now().date()#- timedelta(days=1)
final_current_date = final[final['Datetime'].dt.date == current_date]
final_current_date['Time'] = final_current_date['Datetime'].dt.time
final_current_date.drop(columns=['Datetime'], inplace=True)
fig = go.Figure()
past_data = final_current_date[final_current_date['Flag'] == 'PAST']
fig.add_trace(go.Scatter(x=past_data['Time'], y=past_data['Close'], mode='lines', name='PAST'))
future_data = final_current_date[final_current_date['Flag'] == 'FUTURE']
fig.add_trace(go.Scatter(x=future_data['Time'], y=future_data['Close'], mode='lines', name='FUTURE'))
fig.update_layout(title='Line Chart for PAST and FUTURE Trend', xaxis_title='Time', yaxis_title='Close Price')
fig.update_layout(
    title='Line Chart for PAST and FUTURE Trend',
    xaxis_title='Time',
    yaxis_title='Close Price',
    yaxis=dict(
        tickformat='.2f'
    )
)
fig.show()
